In [1]:
import kaiwu as kw
import numpy as np

def solve_tsp():
    # 定义距离矩阵
    w = np.array([[0, 0, 1, 1, 0],
                  [0, 0, 1, 0, 1],
                  [1, 1, 0, 0, 1],
                  [1, 0, 0, 0, 1],
                  [0, 1, 1, 1, 0]])

    n = w.shape[0]  # 节点数量

    # 创建 QUBO 变量矩阵 (n x n)
    x = kw.qubo.ndarray((n, n), "x", kw.qubo.Binary)

    # 生成边集合和非边集合
    edges = [(u, v) for u in range(n) for v in range(n) if w[u, v] != 0]
    no_edges = [(u, v) for u in range(n) for v in range(n) if w[u, v] == 0]

    # 定义边使用判断函数
    def is_edge_used(x, u, v):
        return kw.qubo.quicksum([x[u, j] * x[v, (j + 1) % n] for j in range(n)])

    # 初始化 QUBO 模型
    qubo_model = kw.qubo.QuboModel()

    # 设置目标函数：最小化路径成本
    path_cost = kw.qubo.quicksum([w[u, v] * is_edge_used(x, u, v) for u, v in edges])
    qubo_model.set_objective(path_cost)

    # 添加约束条件
    # 节点约束：每个节点必须占据一个位置
    qubo_model.add_constraint(x.sum(axis=0) == 1, "node_cons", penalty=5)

    # 位置约束：每个位置必须有一个节点
    qubo_model.add_constraint(x.sum(axis=1) == 1, "pos_cons", penalty=5)

    # 边约束：非连接边不得出现
    qubo_model.add_constraint(
        kw.qubo.quicksum([is_edge_used(x, u, v) for u, v in no_edges]),
        "edge_cons", penalty=5
    )

    # 配置求解器, 需要替换 user_id 和 sdk_code
    optimizer = kw.cim.CIMOptimizer(user_id="YOUR_USER_ID",sdk_code="YOUR_SDK_CODE", task_name="tsp_task")
    optimizer = kw.cim.PrecisionReducer(optimizer, 8)  # 8位精度
    solver = kw.solver.SimpleSolver(optimizer)

    # 求解问题
    sol_dict, qubo_val = solver.solve_qubo(qubo_model)

    # 验证结果
    if sol_dict is not None:
        # 验证结果
        unsatisfied, res_dict = qubo_model.verify_constraint(sol_dict)
        print(f"未满足约束数: {unsatisfied}")
        print(f"约束项值: {res_dict}")

        # 计算路径成本
        path_cost = kw.qubo.get_val(qubo_model.objective, sol_dict)
        print(f"实际路径成本: {path_cost}")

if __name__ == "__main__":
    solve_tsp()

2025-04-28 09:51:00,321 - numexpr.utils - _init_num_threads - 148 - INFO - Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2025-04-28 09:51:00,321 - numexpr.utils - _init_num_threads - 160 - INFO - NumExpr defaulting to 8 threads.


ProxyError: HTTPSConnectionPool(host='open.qboson.com', port=443): Max retries exceeded with url: /sso/access_token/ (Caused by ProxyError('Unable to connect to proxy', SSLError(SSLZeroReturnError(6, 'TLS/SSL connection has been closed (EOF) (_ssl.c:1149)'))))